<a href="https://colab.research.google.com/github/DucBox/Exploratory-Data-Analysis/blob/main/EDA_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import bar_chart_race as bcr

In [ ]:
!pip install bar_chart_race

Load dataset

In [5]:
# Read dataset from csv file
data_file_path = 'covid_jpn_total.csv'
df_raw = pd.read_csv(data_file_path)

In [6]:
# display some data samples
print(df_raw.shape)
df_raw.head(5)

(3526, 19)


,Date,Location,Positive,Tested,Symptomatic,Asymptomatic,Sym-unknown,Hosp_require,Hosp_mild,Hosp_severe,Hosp_unknown,Hosp_waiting,Discharged,Fatal,Vaccinated_1st,Vaccinated_2nd,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th
0,2020-02-06,Domestic,16,132.0,16.0,0.0,0.0,NaN,9.0,0,3.0,0.0,4.0,0,NaN,NaN,NaN,NaN,NaN
1,2020-02-06,Returnee,9,566.0,5.0,4.0,0.0,NaN,3.0,0,2.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-07,Domestic,16,151.0,16.0,0.0,0.0,NaN,12.0,0,0.0,0.0,4.0,0,NaN,NaN,NaN,NaN,NaN
3,2020-02-07,Returnee,9,566.0,6.0,3.0,0.0,NaN,4.0,0,2.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN
4,2020-02-10,Domestic,16,174.0,16.0,0.0,0.0,NaN,7.0,0,0.0,0.0,9.0,0,NaN,NaN,NaN,NaN,NaN


In [7]:
# display data information
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3526 entries, 0 to 3525
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            3526 non-null   object 
 1   Location        3526 non-null   object 
 2   Positive        3526 non-null   int64  
 3   Tested          2838 non-null   float64
 4   Symptomatic     241 non-null    float64
 5   Asymptomatic    241 non-null    float64
 6   Sym-unknown     241 non-null    float64
 7   Hosp_require    2613 non-null   float64
 8   Hosp_mild       241 non-null    float64
 9   Hosp_severe     3526 non-null   int64  
 10  Hosp_unknown    241 non-null    float64
 11  Hosp_waiting    241 non-null    float64
 12  Discharged      2854 non-null   float64
 13  Fatal           3526 non-null   int64  
 14  Vaccinated_1st  2433 non-null   float64
 15  Vaccinated_2nd  2433 non-null   float64
 16  Vaccinated_3rd  2433 non-null   float64
 17  Vaccinated_4th  757 non-null    f

In [8]:
# create a copy version of raw data
df_clean = df_raw.copy()

In [9]:
# remove rows which have all values are null or missing
df_clean.dropna(how="all", inplace=True)

In [10]:
# remove features which have > 50% values are missing
df_clean.drop(['Symptomatic', 'Asymptomatic', 'Sym-unknown', 'Hosp_mild', 'Hosp_unknown',
               'Hosp_waiting', 'Vaccinated_1st', 'Vaccinated_2nd'],
              axis=1,
              inplace=True)

In [11]:
# rename columns for meaningful
df_clean = df_clean.rename(columns={"Discharged":"Cured","Fatal":"Death",
                                    "Hosp_require":"Hospitalization","Hosp_severe":"Severe"})

In [12]:
# convert date feature to Datetime object type
df_clean["Date"] = pd.to_datetime(df_clean["Date"])

In [13]:
# resample data to date level (add the date if it's missed)
df_clean = df_clean.groupby("Location").apply(
    lambda x: x.set_index("Date").resample("D").interpolate(method="linear")
)
# drop location index after grouping
df_clean = df_clean.drop('Location', axis=1).reset_index()

# display 5 first samples
df_clean.head(5)

,Location,Date,Positive,Tested,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th
0,Airport,2020-03-05,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
1,Airport,2020-03-06,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
2,Airport,2020-03-07,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
3,Airport,2020-03-08,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
4,Airport,2020-03-09,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN


In [14]:
# sort data by date and remove data index
df_clean = df_clean.sort_values("Date", ascending=False).reset_index(drop=True)

# display 5 first samples
df_clean.head(5)

,Location,Date,Positive,Tested,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th
0,Returnee,2023-05-08,0.0,829.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN
1,Airport,2023-05-08,24164.0,2356768.0,152.0,0.0,21918.0,8.0,0.0,NaN,NaN
2,Domestic,2023-05-08,33802739.0,73655626.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0
3,Domestic,2023-05-07,33793429.0,73655626.0,530394.0,74.0,20299978.0,74654.0,86485159.0,58557142.0,30571545.0
4,Returnee,2023-05-07,0.0,829.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN


In [ ]:
# impute missing date by using interpolation method
sel = df_clean.columns.isin(["Date", "Location"])
df_clean.loc[:, ~sel] = df_clean.loc[:, ~sel].interpolate("linear").astype(np.int64)

# display 5 first samples
df_clean.head(5)

In [16]:
# create new feature
df_clean['Positive_Tested_Ratio'] = df_clean['Positive']/df_clean['Tested']

# display 5 first samples
df_clean.head(5)

,Location,Date,Positive,Tested,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th,Positive_Tested_Ratio
0,Returnee,2023-05-08,0.0,829.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,0.000000
1,Airport,2023-05-08,24164.0,2356768.0,152.0,0.0,21918.0,8.0,0.0,NaN,NaN,0.010253
2,Domestic,2023-05-08,33802739.0,73655626.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0,0.458929
3,Domestic,2023-05-07,33793429.0,73655626.0,530394.0,74.0,20299978.0,74654.0,86485159.0,58557142.0,30571545.0,0.458803
4,Returnee,2023-05-07,0.0,829.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,0.000000


In [17]:
# drop 'Tested' feature
df_clean.drop(['Tested'], axis=1, inplace=True)

# display 5 first samples
df_clean.head(5)

,Location,Date,Positive,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th,Positive_Tested_Ratio
0,Returnee,2023-05-08,0.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,0.000000
1,Airport,2023-05-08,24164.0,152.0,0.0,21918.0,8.0,0.0,NaN,NaN,0.010253
2,Domestic,2023-05-08,33802739.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0,0.458929
3,Domestic,2023-05-07,33793429.0,530394.0,74.0,20299978.0,74654.0,86485159.0,58557142.0,30571545.0,0.458803
4,Returnee,2023-05-07,0.0,0.0,0.0,15.0,0.0,0.0,NaN,NaN,0.000000


EDA

In [18]:
# create pivot table
df_pos = df_clean.pivot_table(
    index="Date", columns="Location", values="Positive", aggfunc="last"
)

# display 5 last samples
df_pos.tail(5)

Location,Airport,Domestic,Returnee
Date,,,
2023-05-04,24134.0,33766957.0,0.0
2023-05-05,24138.0,33772764.0,0.0
2023-05-06,24143.0,33778993.0,0.0
2023-05-07,24147.0,33793429.0,0.0
2023-05-08,24164.0,33802739.0,0.0


In [19]:
# Data in 2021-08-13 seem to be abnormal. So, remove it.
df_pos.drop(["2021-08-13"], inplace=True)

In [20]:
# draw line chart
fig = px.line(df_pos,
              y=['Airport', 'Domestic', 'Returnee'],
              labels={"index":"Date", "value":"Number of positive people", "variable":"Locations"})
fig.show()

Compare change in number of dead cases of all location over time

In [21]:
df_death = df_clean.pivot_table(
    index="Date", columns="Location", values="Death", aggfunc="last"
)

# display 5 last samples
df_death.tail(5)

Location,Airport,Domestic,Returnee
Date,,,
2023-05-04,8.0,74614.0,0.0
2023-05-05,8.0,74633.0,0.0
2023-05-06,8.0,74645.0,0.0
2023-05-07,8.0,74654.0,0.0
2023-05-08,8.0,74669.0,0.0


In [22]:
# draw line chart
fig = px.line(df_death,
              y=['Airport', 'Domestic', 'Returnee'],
              labels={"index":"Date", "value":"Number of dead people", "variable":"Locations"})
fig.show()

Situation In Japan

In [23]:
# select data only in domestic Japan
df_jp = df_clean[df_clean['Location'] == 'Domestic'].drop(['Location'], axis=1)

# display 5 last samples
df_jp.head()

,Date,Positive,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th,Positive_Tested_Ratio
2,2023-05-08,33802739.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0,0.458929
3,2023-05-07,33793429.0,530394.0,74.0,20299978.0,74654.0,86485159.0,58557142.0,30571545.0,0.458803
8,2023-05-06,33778993.0,530394.0,74.0,20299978.0,74645.0,86485159.0,58557142.0,30571545.0,0.458607
11,2023-05-05,33772764.0,530394.0,73.0,20299978.0,74633.0,86485159.0,58557142.0,30571545.0,0.458523
12,2023-05-04,33766957.0,530394.0,71.0,20299978.0,74614.0,86485159.0,58557142.0,30571545.0,0.458444


In [24]:
# set Date feature as index
df_jp = df_jp.set_index(['Date'])

# display 5 first samples
df_jp.head()

,Positive,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th,Positive_Tested_Ratio
Date,,,,,,,,,
2023-05-08,33802739.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0,0.458929
2023-05-07,33793429.0,530394.0,74.0,20299978.0,74654.0,86485159.0,58557142.0,30571545.0,0.458803
2023-05-06,33778993.0,530394.0,74.0,20299978.0,74645.0,86485159.0,58557142.0,30571545.0,0.458607
2023-05-05,33772764.0,530394.0,73.0,20299978.0,74633.0,86485159.0,58557142.0,30571545.0,0.458523
2023-05-04,33766957.0,530394.0,71.0,20299978.0,74614.0,86485159.0,58557142.0,30571545.0,0.458444


In [25]:
# Data in 2021-08-13 seem to be abnormal. So, remove it.
df_jp.drop(["2021-08-13"], inplace=True)

In [26]:
# draw line chart
fig = px.line(df_jp,
             y=df_jp.columns.to_list(),
             title='Compare change in number of some categories over time',
             labels={"index":"Date", "value":"Number of people", "variable":""})
fig.show()

In [ ]:
# sort data by feature 'Date' ascending
df_jp = df_jp.sort_values(['Date'], ascending=True)

# draw bar char in video type
bcr.bar_chart_race(df=df_jp,
                   n_bars=5,
                   figsize=(4,2),
                   dpi=250,
                   title='Compare change in number of some categories over time',
                   title_size=8,
                   bar_label_size=6,
                   tick_label_size=6)

In [28]:
# draw bar chart
fig = px.bar(df_jp,
             x=df_jp.index,
             y='Death',
             color='Death',
             title="Change in number of dead people over time",
             labels={"Death":"Number of dead people"})
fig.show()

In [29]:
fig = px.bar(df_jp,
             x=df_jp.index,
             y='Hospitalization',
             color='Hospitalization',
             labels={"index":"Number of Tests","value":"Date"},
             title="Change in number of people who need to hospitalize")
fig.show()

In [30]:
fig = px.bar(df_jp,
             x=df_jp.index,
             y='Positive_Tested_Ratio',
             color='Positive_Tested_Ratio',
             labels={"index":"Number of Tests","value":"Date"},
             title="Change in number of people who need to hospitalize")
fig.show()

In [31]:
df_cur = df_jp.tail(1)
df_cur

,Positive,Hospitalization,Severe,Cured,Death,Vaccinated_3rd,Vaccinated_4th,Vaccinated_5th,Positive_Tested_Ratio
Date,,,,,,,,,
2023-05-08,33802739.0,530394.0,80.0,20299978.0,74669.0,86492902.0,58570394.0,30580204.0,0.458929


In [32]:
# draw pie chart
fig = px.pie(df_jp,
             values=np.array(df_cur)[0],
             names=df_cur.columns.to_list(),
             title='Current Situation in Japan (2021-08-16)',
             color_discrete_sequence=px.colors.sequential.Rainbow)
fig.show()